In [1]:
import os
import time
import queue
import requests
import datetime
import threading
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# set the year we need & get all teams in NBA

In [26]:
#create a list from 2007-2018
year = []
# for i in range(2007,2019):
for i in range(2019,2020):
    year.append(i)
#Get 30 teams in NBA
url = 'https://www.basketball-reference.com/'
table = pd.read_html(url)
for i in range(len(table)):
    if len(table[i]) >= 15:
        if i == 0:
            East = table[i]
        else:
            West = table[i]
East = East.East
East = list(East)
West = West.West
West = list(West)
nbateam = []
nbateam.append(East)
nbateam.append(West)
#filter other words in both east and west
for t in nbateam:
    for i in range(len(t)):
        count = 0
        team = ''
        for j in t[i]:
            count = count + 1
            team = team + j
            if count == 3:
                t[i] = team
                break

In [29]:
path = 'C:\\Users\\ecom\\Desktop\\NBAData\\2019'
if not os.path.isdir(path):
    os.mkdir(path)
for i in range(len(nbateam)):
    if i == 0:
        side = 'East'
    else:
        side = 'West'
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\2019\\' + side
    if not os.path.isdir(path):
        os.mkdir(path)
    for j in nbateam[i]:
        path = 'C:\\Users\\ecom\\Desktop\\NBAData\\2019\\' + side + '\\' + j
        if not os.path.isdir(path):
            os.mkdir(path)

# get 2007~2018 nba players list in each team

In [21]:
tStart = time.time()
Players1 = pd.DataFrame(columns=['name', 'webname', 'team', 'year', 'side', 'website'])
path = 'C:\\Users\\ecom\\Desktop\\NBAData'
if not os.path.isdir(path):
    os.mkdir(path)

for i in year:
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i)
    if not os.path.isdir(path):
        os.mkdir(path)
        
    for t in range(len(nbateam)):
        if t == 0:
            s = 'East'
        elif t == 1:
            s = 'West'
        path = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i) + '\\' + s
        if not os.path.isdir(path):
            os.mkdir(path)
        path1 = path

        for j in nbateam[t]:
            
            #change teamname
            teamname = j
            if i <= 2012 and j == 'BRK':
                j = 'NJN'
            elif i <= 2014 and j == 'CHO':
                j = 'CHA'
            elif i == 2007 and j == 'NOP':
                j = 'NOK'
            elif i >= 2008 and i <= 2013 and j == 'NOP':
                j = 'NOH'
            elif i <= 2008 and j == 'OKC':
                j = 'SEA'
                
            path = path1 + '\\' + teamname
            if not os.path.isdir(path):
                os.mkdir(path)
            webplayerlist = []
            url = 'https://www.basketball-reference.com/teams/' + j + '/' + str(i) + '.html'
                
            # print(url)
            table = pd.read_html(url)
            playerlist = list(table[0].Player)
            for w in range(len(playerlist)):
                wor = ''
                for z in playerlist[w]:
                    if z == '(':
                        break
                    else:
                        wor = wor + z
                playerlist[w] = wor
            playerlist.sort()
            tm = []
            yr = []
            sd = []
            website = []

            pageSource = requests.get(url)
            sp = BeautifulSoup(pageSource.text, "lxml")
            table = sp.select('td.left')
            na = []
            np = []
            for f in range(len(table)):
                ul = table[f].get('data-append-csv')
                pn = table[f].text
                na.append(ul)
                np.append(pn)

            while None in na:
                na.remove(None)
            na1 = list(set(na))
            na1.sort(key=na.index)

            cal = 0
            np1 = []
            for v in np:
                if cal == 0:
                    np1.append(v)
                    cal = cal + 1
                elif cal != 0 and cal % 3 == 0:
                    np1.append(v)
                    cal = cal + 1
                else:
                    cal = cal + 1
                    continue
            
            for nu in range(len(na1)):
                tm.append(j)
                yr.append(str(i))
                sd.append(s)
            for wb in na1:
                for wba in wb:
                    letter = wba
                    break
                url1 = 'https://www.basketball-reference.com/players/' + letter + '/' + wb + '/gamelog/' + str(i) + '/'
                website.append(url1)
            
            Players = pd.DataFrame(columns=['name', 'webname', 'team', 'year', 'side', 'website'])
            Players.name = np1
            Players.webname = na1
            Players.team = tm
            Players.year = yr
            Players.side = sd
            Players.website = website
            Players1 = Players1.append(Players, ignore_index=True)
plist = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\playerList.xlsx')
plist = plist.append(Players1)
plist = plist.reset_index(drop=True)
plist.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\playerList.xlsx')
tEnd = time.time()
print('Cost %f s' %(tEnd - tStart))
#almost 20 mins

ValueError: Length of values does not match length of index

In [27]:
df = pd.DataFrame(columns=['team', 'side','name', 'website'])
for side in range(len(nbateam)):
    if side == 0:
        teamside = 'East'
    else:
        teamside = 'West'
    for teams in nbateam[side]:
        print(teams)
        team = []
        website = []
        si = []
        url1 = 'https://www.basketball-reference.com/teams/' + teams + '/2019.html'
        table = pd.read_html(url1)
        playerlist = list(table[0].Player)
        for w in range(len(playerlist)):
            wor = ''
            for z in playerlist[w]:
                if z == '(':
                    break
                else:
                    wor = wor + z
            playerlist[w] = wor
        pageSource = requests.get(url1)
        sp = BeautifulSoup(pageSource.text, "lxml")
        table = sp.select('tbody > tr')
        sel = []
        for i in range(len(table)):
            ao = table[i].select('td.left')
            sel.append(ao[0].select('a')[0].get('href'))
        for i in sel:
            url1 = 'https://www.basketball-reference.com' + i
            website.append(url1)
            team.append(teams)
            si.append(teamside)
        Players = pd.DataFrame(columns=['team', 'side', 'name', 'website'])
        Players.team = team
        Players.side = si
        Players.name = playerlist
        Players.website = website
        df = df.append(Players, ignore_index=True)
    df.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\2019_playerList.xlsx')

TOR
MIL
IND
BOS
PHI
CHO
DET
BRK
MIA
ORL
ATL
NYK
WAS
CHI
CLE
GSW
DEN
POR
SAS
MEM
SAC
LAC
OKC
NOP
UTA
MIN
LAL
HOU
DAL
PHO


In [28]:
dfplayerlist = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\2019_playerList.xlsx')
web = dfplayerlist.website
for i in range(len(web)):
    web[i] = web[i].replace('.html', '/gamelog/2019/')
    print(web[i])
dfplayerlist.website = web
dfplayerlist.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\2019_playerList.xlsx')

https://www.basketball-reference.com/players/v/valanjo01/gamelog/2019/
https://www.basketball-reference.com/players/w/wrighde01/gamelog/2019/
https://www.basketball-reference.com/players/l/leonaka01/gamelog/2019/
https://www.basketball-reference.com/players/g/greenda02/gamelog/2019/
https://www.basketball-reference.com/players/r/richama01/gamelog/2019/
https://www.basketball-reference.com/players/s/siakapa01/gamelog/2019/
https://www.basketball-reference.com/players/a/anunoog01/gamelog/2019/
https://www.basketball-reference.com/players/m/milescj01/gamelog/2019/
https://www.basketball-reference.com/players/l/lowryky01/gamelog/2019/
https://www.basketball-reference.com/players/i/ibakase01/gamelog/2019/
https://www.basketball-reference.com/players/p/powelno01/gamelog/2019/
https://www.basketball-reference.com/players/v/vanvlfr01/gamelog/2019/
https://www.basketball-reference.com/players/b/brownlo01/gamelog/2019/
https://www.basketball-reference.com/players/b/bouchch01/gamelog/2019/
https:

In [30]:
erroryear = []
errorteam = []
errorname = []
errortable = []
nowteam = ''
dfplayerlist = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\2019_playerList.xlsx')
for urlnum in range(len(dfplayerlist.website)):
    try:
        url = dfplayerlist.website[urlnum]
        tables = pd.read_html(url)
        pside = dfplayerlist.side[urlnum]
        pteam = dfplayerlist.team[urlnum]
        pname = dfplayerlist.name[urlnum]
        
        if pteam != nowteam:
            print(pteam)
            nowteam = pteam
        lntable = 0
        for ts in tables:
            if len(ts) > lntable:
                lntable = len(ts)
                table = ts
        gcol = []
        count = 0
        for c in table['G']:
            if c == 'G':
                gcol.append(count)
            count = count + 1
        ctable = table.drop(gcol)
        ctable = ctable.reset_index()
        ctable.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\2019\\' + pside + '\\' + pteam + '\\' + pname + '.xlsx')
    except Exception as e:
        erroryear.append('2019')
        errorteam.append(pteam)
        errorname.append(pname)
        errortable.append(url)
        continue

TOR
MIL
IND
BOS
PHI
CHO
DET
BRK
MIA
ORL
ATL
NYK
WAS
CHI
CLE
GSW
DEN
POR
SAS
MEM
SAC
LAC
OKC
NOP
UTA
MIN
LAL
HOU
DAL
PHO


# get each player's gamelog from season 2007 to 2018

In [5]:
erroryear = []
errorteam = []
errorname = []
errortable = []
urlQueue = queue.Queue()
dfplayerlist = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\playerList.xlsx')
web = list(dfplayerlist.website)
for url in web:
    urlQueue.put(url)
def getpersonaldata(n):
    while True:
        try:
            url = n.get_nowait()
            i = n.qsize()
        except Exception as e:
            break
        try:
            tables = pd.read_html(url)
            for j in range(len(dfplayerlist.website)):
                if dfplayerlist.website[j] == url:
                    pyear = dfplayerlist.year[j]
                    pside = dfplayerlist.side[j]
                    pteam = dfplayerlist.team[j]
                    pname = dfplayerlist.name[j]
            lntable = 0
            for ts in tables:
                if len(ts) > lntable:
                    lntable = len(ts)
                    table = ts
            gcol = []
            count = 0
            for c in table['G']:
                if c == 'G':
                    gcol.append(count)
                count = count + 1
            ctable = table.drop(gcol)
            ctable = ctable.reset_index()
            ctable.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\' + str(pyear) + '\\' + pside + '\\' + pteam + '\\' + pname + '.xlsx')
        except Exception as e:
            erroryear.append(pyear)
            errorteam.append(pteam)
            errorname.append(pname)
            errortable.append(url)
            continue

if __name__=='__main__':
    time_start=time.time()
    thr = []
    thrnum = 5
    for i in range(thrnum):
        t = threading.Thread(target=getpersonaldata, args=(urlQueue,))
        thr.append(t)
    for t in thr:
        t.start()
    for t in thr:
        t.join()

if len(erroryear) != 0:
    errordic = {'year' : erroryear, 'team' : errorteam, 'name' : errorname, 'website' : errortable}
    error = pd.DataFrame(errordic)
    error.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\error.xlsx')
time_end=time.time()
cost = time_end - time_start
sec = cost % 60
last = int(cost / 60)
minites = last % 60
hour = int(last / 60)
print('time cost ' + str(hour) + ' h ' + str(minites) + ' m ' + str(sec) + ' s ')

time cost 1 h 25 m 29.120286464691162 s 


# Rebuild NBA data folder (If needed)

In [4]:
#rebuild folder
tStart = time.time()

path = 'C:\\Users\\ecom\\Desktop\\NBAData'
if not os.path.isdir(path):
    os.mkdir(path)

for i in year:
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i)
    if not os.path.isdir(path):
        os.mkdir(path)
        
    for t in range(len(nbateam)):
        if t == 0:
            s = 'East'
        elif t == 1:
            s = 'West'
        path = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i) + '\\' + s
        if not os.path.isdir(path):
            os.mkdir(path)
        path1 = path

        for j in nbateam[t]:
            
            #change teamname
            teamname = j
            if i <= 2012 and j == 'BRK':
                j = 'NJN'
            elif i <= 2014 and j == 'CHO':
                j = 'CHA'
            elif i == 2007 and j == 'NOP':
                j = 'NOK'
            elif i >= 2008 and i <= 2013 and j == 'NOP':
                j = 'NOH'
            elif i <= 2008 and j == 'OKC':
                j = 'SEA'
                
            
            path = path1 + '\\' + teamname
            if not os.path.isdir(path):
                os.mkdir(path)

tEnd = time.time()
print('Cost %f s' %(tEnd - tStart))
#almost 20 mins

Cost 0.134977 s


# get 2017, 2018 game schedual

In [3]:
month = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
path = 'C:\\Users\\ecom\\Desktop\\NBAData\\schedual'
if not os.path.isdir(path):
    os.mkdir(path)
for i in range(2017,2020):
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i)
    if not os.path.isdir(path):
        os.mkdir(path)
    for j in month:
        url = 'https://www.basketball-reference.com/leagues/NBA_' + str(i) + '_games-' + j + '.html'
        table = pd.read_html(url)
        
        for k in range(len(table)):
            dfs = table[k]
            if j == 'april':
                pof = dfs.loc[dfs['Date'] == 'Playoffs',].index
                pof1 = pof[0]
                dfs = dfs.drop(range(pof1,len(dfs.loc[pof1:]) + pof1))
            
            # process date format
            d = dfs[['Date']].values
            duniq = np.unique(d)
            for t in range(len(duniq)):
                count = 0
                dword = ''
                for u in duniq[t]:
                    count = count + 1
                    if dword == 'Oct':
                        dword = '10,'
                    elif dword == 'Nov':
                        dword = '11,'
                    elif dword == 'Dec':
                        dword = '12,'
                    elif dword == 'Jan':
                        dword = '1,'
                    elif dword == 'Feb':
                        dword = '2,'
                    elif dword == 'Mar':
                        dword = '3,'
                    elif dword == 'Apr':
                        dword = '4,'
                    if count >= 6:
                        dword = dword + u
                duniq[t] = dword
            date_time_list = []
            for x in duniq:
                date_time = datetime.datetime.strptime(x,'%m, %d, %Y')
                date_time = date_time.date()
                date_time_list.append(str(date_time))
            dicd = {}
            for t in range(len(duniq)):
                dicd[np.unique(d)[t]] = date_time_list[t]
            newd = []
            for t in d:
                newd.append(dicd.get(t[0]))
            dfs[['Date']] = newd
            
            # change team name
            dfs = dfs.drop(['Start (ET)', ' ', ' .1', 'Attend.', 'Notes'], axis=1)
            teamlist = dfs[['Visitor/Neutral']].values
            teamlist1 = dfs[['Home/Neutral']].values
            teamlist = np.append(teamlist, teamlist1)
            teamlist = np.unique(teamlist)
            side = ['East', 'West']
            bothteam = []
            for t in side:
                teampath = 'C:\\Users\\ecom\\Desktop\\NBAData\\gamelog\\2017\\' + t
                halfside = os.listdir(teampath)
                for u in halfside:
                    bothteam.append(u)
            bothteam.sort()
            dic = {}
            for t in range(30):
                dic[teamlist[t]] = bothteam[t]
            dic['Charlotte Hornets'] = 'CHO'
            dic['Chicago Bulls'] = 'CHI'
            nat = ['Visitor/Neutral', 'Home/Neutral']
            for t in nat:
                sc = []
                for u in dfs[[t]].values:
                    sc.append(dic.get(u[0]))
                dfs[[t]] = sc
            
            path = 'C:\\Users\\ecom\\Desktop\\NBAData\\schedual\\' + str(i) + '\\' + j + '.xlsx'
            dfs.to_excel(path)

IndexError: index 0 is out of bounds for axis 0 with size 0

# test

In [ ]:
erroryear = []
errorteam = []
errorname = []

tStart = time.time()
for i in year:
    for t in range(len(nbateam)):
        if t == 0:
            s = 'East'
        elif t == 1:
            s = 'West'
        print(str(i) + '-' + s + ' is parsing...')
        for j in nbateam[t]:
            dfplayer = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\playerlist\\' + str(i) + '\\' + s + '\\' + j + '\\' + j + '.xlsx')
            plist = list(dfplayer.webname)
            for k in plist:
                for a in k:
                    letter = a
                    break
                try:
                    url = 'https://www.basketball-reference.com/players/' + letter + '/' + k + '/gamelog/' + str(i) + '/'
                    tables = pd.read_html(url)
                except:
                    erroryear.append(str(i))
                    errorteam.append(j)
                    errorname.append(k)
                    continue
                try:
                    for n in tables:
                        if len(n) >= 86:
                            table = n
                    gcol = []
                    count = 0
                    for c in table['G']:
                        if c == 'G':
                            gcol.append(count)
                        count = count + 1
                    ctable = table.drop(gcol)
                    ctable = ctable.reset_index()
                    ctable.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i) + '\\' + s + '\\' + j + '\\' + k + '.xlsx')
                except:
                    continue
errordic = {'year' : erroryear, 'team' : errorteam, 'name' : errorname}
error = pd.DataFrame(errordic)
error.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\error.xlsx')
tEnd = time.time()
print('Cost %f s' %(tEnd - tStart))
#almost 7 hours